# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [705]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [706]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
print("Number of files is {}".format(len(file_path_list)))

/home/workspace
Number of files is 30


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [707]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
               

In [708]:
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16])) 

In [709]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Creating denormalized table in Apache Cassanadra. 

The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [710]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [711]:
#Creating a keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [712]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

In [713]:
def process_query(session,query,return_rows=False):
    """
    Process a cassandra query
    Inputs:
        session(object): current cassandra session
        query(string): query to be processed
        rows(bool): query result
    
    Returns:
        query result if return_rows is True
    """
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)
    if return_rows == True:
        return rows
    return None

In [714]:
def insert_data(filename,session,query,indices):
    """
    using pipeline insert data into cassandra table
    inputs:
        file(str): CSV filename
        session(string): current cassandra session
        query(string):the insert query statement
        indices(list): the indexes of the data to insert
    """
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            session.execute(query,(tuple(line[x] for x in indices)))

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [715]:
query = "CREATE TABLE IF NOT EXISTS music_library"

#combining sessionId and itemInSession in this table would provide for unique primary key
query = query + "(sessionId text, itemInSession text, artist text, song text, length text,\
                PRIMARY KEY (sessionId, itemInSession))"
process_query(session,query,False)

In [716]:
file = 'event_datafile_new.csv'

query = "INSERT INTO music_library (sessionId, itemInSession, artist, song, length)"
query = query + " VALUES (%s, %s, %s, %s, %s)"                      
#Assign which column element should be assigned for each column in the INSERT statement.
indices = [8,3,0,9,5]                       
insert_data(file,session,query,indices)

# [0:'artist',
# 1:'firstName',
# 2:'gender',
# 3:'itemInSession',
# 4:'lastName',
# 5:'length',
# 6:'level',
# 7:'location',
# 8:'sessionId',
# 9:'song',
# 10:'userId']  

In [717]:
query = "select artist, song, length from music_library WHERE sessionId = '338' AND itemInSession = '4'"
rows = process_query(session,query,True)

query_result = []
for row in rows:
        query_result.append((row.artist, row.song, row.length))

df = pd.DataFrame(query_result,columns = ['artist','song','length'])
df.head()    

artist                             song    length
0  Faithless  Music Matters (Mark Knight Dub)  495.3073

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [718]:
query = "CREATE TABLE IF NOT EXISTS music_library2"
query = query + "(userId text, sessionId text, itemInSession text, artist text, song text, firstName text,  lastName text,\
                PRIMARY KEY ((userId, sessionId), itemInSession) ) WITH CLUSTERING ORDER BY (itemInSession DESC )"
process_query(session,query)

#made a primary key of partition key and clustering column andf then sorted on intemInSession 

In [719]:
query = "INSERT INTO music_library2 (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"        
indices = [10,8,3,0,9,1,4]                       
insert_data(file,session,query,indices)   

In [720]:
query = "select song, firstName, lastName, userId, itemInSession, artist from music_library2 WHERE userId = '10'\
        AND sessionId = '182' "
rows = process_query(session,query,True)

query_result = []
for row in rows:
    query_result.append((row.artist, row.song,row.firstname, row.lastname, row.iteminsession))

df = pd.DataFrame(query_result,columns = ['artist','song','firstname','lastname','iteminsession'])
df.head()

artist                                               song  \
0      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   
1  Sebastien Tellier                                          Kilometer   
2       Three Drives                                        Greece 2000   
3   Down To The Bone                                 Keep On Keepin' On   

  firstname lastname iteminsession  
0    Sylvie     Cruz             3  
1    Sylvie     Cruz             2  
2    Sylvie     Cruz             1  
3    Sylvie     Cruz             0

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [721]:
query = "CREATE TABLE IF NOT EXISTS music_library3"
query = query + "(song text, userId text, firstName text, lastName text , PRIMARY KEY (song, userId))"
process_query(session,query)

#added userId to the primary key to provide for unique rows

In [722]:
query = "INSERT INTO music_library3 (song, userId, firstName, lastName)"
query = query + " VALUES (%s, %s, %s, %s)"       
indices = [9,10,1,4]                       
insert_data(file,session,query,indices)            

In [723]:
query = "SELECT song, firstname, lastName FROM music_library3 WHERE song = 'All Hands Against His Own' "

rows = process_query(session,query,True)
        
query_result = []
for row in rows:
    query_result.append((row.firstname, row.lastname,row.song))

df = pd.DataFrame(query_result,columns = ['firstname','lastname','song'])
df.head()

firstname lastname                       song
0  Jacqueline    Lynch  All Hands Against His Own
1       Tegan   Levine  All Hands Against His Own
2        Sara  Johnson  All Hands Against His Own

### Drop the tables before closing out the sessions

In [724]:
query = "drop table music_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table music_library2"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table music_library3"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [725]:
session.shutdown()
cluster.shutdown()